# 基于LLM微调的数学推理任务

In [1]:
import os 
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader
from torch.nn.functional import one_hot
from sklearn.metrics import accuracy_score, f1_score
from evaluate import load
from datasets import load_dataset

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

2024-12-07 20:31:05.651842: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-07 20:31:07.237439: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# # 设置代理
# import subprocess
# import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

In [6]:
from datasets import load_dataset



test_dataset = load_dataset('json', data_files='./MATH_test.jsonl', split='train')
train_dataset = load_dataset('json', data_files='./MATH_train.jsonl', split='train')



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
def get_dataloader(prompt, batch_size=1):
    def preprocess(dataset, shuffle=True):
        def collate_fn(batch):
            questions =  [[{"role": "user", "content": prompt.format(text=item['problem'])}] for item in batch]
            answers = [item["solution"] for item in batch]
            return questions, answers
        return DataLoader(
            dataset,
            shuffle=shuffle,
            batch_size=batch_size,
            collate_fn=collate_fn,
        )
    testset = test_dataset
    return preprocess(testset, shuffle=False)

In [15]:
#  ## setup math_equivalence ： 在math工作目录中 pip install .
# import math_equivalence
from modelscope import AutoModelForCausalLM, AutoTokenizer
import torch

from tqdm import tqdm
import json

def eval_process(model_name, tokenizer_name, prompt, batch_size=1, ratio=1):
    # 检查是否有可用的 GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # 加载模型和分词器到 GPU
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",         # 如果有多个 GPU，可以自动分配
        cache_dir='/autodl-tcachemp/'
    ).to(device)  # 确保模型在 GPU 上

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    # 加载数据
    testloader = get_dataloader(prompt, batch_size)
    print(testloader)
    
    answers = []
    replies = []

    # 遍历数据集
    for idx, (texts, truths) in tqdm(enumerate(testloader), total=int(len(testloader) * ratio)):
        if idx >= int(len(testloader) * ratio):
            break

        # 应用模板并生成输入
        texts = [tokenizer.apply_chat_template(
            text,
            tokenize=False,
            add_generation_prompt=True
        ) for text in texts]

        # 将输入张量移到 GPU 上
        model_inputs = tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            padding_side='left'
        ).to(device)

        # 在 GPU 上生成文本
        with torch.no_grad():  # 禁用梯度计算，节省显存
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=256
            )

        # 截取生成的结果
        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        # 解码生成的结果
        responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        replies.extend(responses)
        answers.extend(truths)
        print(len(replies))
    # 计算正确率
    # total = 0
    # correct = 0
    # for reply, answer in zip(replies, answers):
    #     total += 1
    #     if math_equivalence.is_equiv(reply, answer) :
    #         correct += 1
    # print(f"正确率: {(correct / total) * 100: .2f}%")
    results = [{"answer": a, "reply": r} for a, r in zip(answers, replies)]
    output_file = "results_lora.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    print(f"Results saved to {output_file}")

    return replies,answers

In [16]:
model_name = "output_lora/qwen2_5-0_5b-instruct/v0-20241207-203247/checkpoint-464"
tokenizer_name = "Qwen/Qwen2.5-0.5B-Instruct"
torch.cuda.empty_cache()
prompt = '''You are a highly skilled mathematician capable of solving complex grade-school math problems step by step. Please read the problem carefully and solve it with clear reasoning. Follow these instructions:

- Identify the known information and the question being asked.
- Break the solution into logical steps, providing clear explanations for each.
- Show all calculations and intermediate results.
- Conclude with a final answer.

Output your response in the following format:
[Explanation and calculations]
#### [Final numerical answer]   

Here is the problem:
{text}'''
eval_process(model_name, tokenizer_name, prompt, batch_size=32, ratio=1)

Using device: cuda


[INFO:modelscope] Creating symbolic link /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2.5-0.5B-Instruct.


  1%|          | 1/157 [00:17<44:52, 17.26s/it]

32


  1%|▏         | 2/157 [00:37<48:36, 18.82s/it]

...

 99%|█████████▉| 156/157 [47:54<00:17, 17.10s/it]

4992


100%|██████████| 157/157 [48:09<00:00, 18.40s/it]

5000
Results saved to results_lora.json


(['We can rewrite the denominator as $(x+3)(x-2)$.  Thus, there are no vertical asymptotes where $x= \\pm 3$ or $x=-2$, so we look at the factors that are not equal to zero: $(x+2)$ and $(x-3)$.  These factors correspond to two vertical asymptotes.  Therefore, the number of vertical asymptotes is $\\boxed{2}$.',
  'First we calculate $120\\%$ of 30: \\begin{align*}\n120\\%&=120\\div100\\\\\n&=\\frac{6}{5}\\\\\n&=1.2\\\\\n&=120\\%\\text{ of }30\\\\\n&=1.2\\times30\\\\\n&=36.\n\\end{align*}Next we calculate $130\\%$ of 20: \\begin{align*}\n130\\%&=130\\div100\\\\\n&=\\frac{7}{5}\\\\\n&=\\frac{35}{25}\\\\\n&=\\frac{7}{5}\\times\\frac{25}{25}\\\\\n&=\\frac{175}{125}\\\\\n&=\\frac{7}{5}\\times\\frac{25}{25}\\\\\n&=\\frac{7}{5}\\times4\\\\\n&=8.\n\\end{align*}Finally, we find the positive difference between 36 and 8: \\begin{align*}\n36-8&=28.\n\\end{align',
  'Since $\\lceil x\\rceil$ represents the smallest integer greater than or equal to $x$, we have $x - \\lceil x\\rceil = \\dfrac{23}{7

In [9]:
# 导入swift框架进行微调
from swift.llm import (
    DatasetName, InferArguments, ModelType, SftArguments,
    infer_main, sft_main, app_ui_main
)

[INFO:swift] Successfully registered `/usr/local/lib/python3.10/site-packages/swift/llm/data/dataset_info.json`


In [10]:
model_type = ModelType.qwen2_5_0_5b_instruct
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())
sft_args = SftArguments(
    model_type=model_type,
    dataset=['MATH_train_fineture.jsonl'],
    output_dir='output_lora',
    max_length=4096,
    # sft_type = SftType.full,
    
    )
result = sft_main(sft_args)
last_model_checkpoint = result['last_model_checkpoint']

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

[INFO:swift] Setting template_type: qwen2_5
[INFO:swift] Setting args.lazy_tokenize: False
[INFO:swift] Setting args.dataloader_num_workers: 1
[INFO:swift] output_dir: /mnt/workspace/output_lora/qwen2_5-0_5b-instruct/v0-20241207-203247
[INFO:swift] Start time of running main: 2024-12-07 20:32:47.412100
[INFO:swift] args: SftArguments(model_type='qwen2_5-0_5b-instruct', model_id_or_path='qwen/Qwen2.5-0.5B-Instruct', model_revision='master', full_determinism=False, sft_type='lora', freeze_parameters=[], freeze_vit=False, freeze_parameters_ratio=0.0, additional_trainable_parameters=[], tuner_backend='peft', template_type='qwen2_5', output_dir='/mnt/workspace/output_lora/qwen2_5-0_5b-instruct/v0-20241207-203247', add_output_dir_suffix=True, ddp_backend=None, ddp_find_unused_parameters=None, ddp_broadcast_buffers=None, ddp_timeout=1800, seed=42, resume_from_checkpoint=None, resume_only_model=False, ignore_data_skip=False, dtype='bf16', packing=False, train_backend='transformers', tp=1, pp=1

device_count: 1
rank: -1, local_rank: -1, world_size: 1, local_world_size: 1


[WARNING:modelscope] Using branch: master as version is unstable, use with caution


[INFO:modelscope] Creating symbolic link /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2.5-0.5B-Instruct.
[INFO:swift] Loading the model using model_dir: /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct
[INFO:swift] model_kwargs: {'device_map': 'cuda:0'}
[INFO:swift] model.max_model_len: 32768
[INFO:swift] model.hf_device_map: {'': device(type='cuda', index=0)}
[INFO:swift] model_config: Qwen2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "/mnt/workspace/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24

...

{'eval_loss': 0.70992064, 'eval_acc': 0.81676964, 'eval_runtime': 11.301, 'eval_samples_per_second': 6.637, 'eval_steps_per_second': 6.637, 'epoch': 0.97, 'global_step/max_steps': '450/464', 'percentage': '96.98%', 'elapsed_time': '52m 29s', 'remaining_time': '1m 37s'}
{'loss': 0.73779354, 'acc': 0.77980127, 'grad_norm': 0.51511723, 'learning_rate': 1e-07, 'memory(GiB)': 6.13, 'train_speed(iter/s)': 0.142879, 'epoch': 0.98, 'global_step/max_steps': '455/464', 'percentage': '98.06%', 'elapsed_time': '53m 3s', 'remaining_time': '1m 2s'}
{'loss': 0.74050436, 'acc': 0.7876061, 'grad_norm': 0.52593786, 'learning_rate': 2e-08, 'memory(GiB)': 6.13, 'train_speed(iter/s)': 0.143038, 'epoch': 0.99, 'global_step/max_steps': '460/464', 'percentage': '99.14%', 'elapsed_time': '53m 35s', 'remaining_time': '27s'}



Val:   0%|          | 0/75 [00:00<?, ?it/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[INFO:swift] Saving model checkpoint to /mnt/workspace/output_lora/qwen2_5-0_5b-instruct/v0-20241207-203247/checkpoint-464
[INFO:swift] last_model_checkpoint: /mnt/workspace/output_lora/qwen2_5-0_5b-instruct/v0-20241207-203247/checkpoint-464


{'eval_loss': 0.70946109, 'eval_acc': 0.81659312, 'eval_runtime': 11.2344, 'eval_samples_per_second': 6.676, 'eval_steps_per_second': 6.676, 'epoch': 1.0, 'global_step/max_steps': '464/464', 'percentage': '100.00%', 'elapsed_time': '54m 14s', 'remaining_time': '0s'}
{'train_runtime': 3254.2059, 'train_samples_per_second': 2.282, 'train_steps_per_second': 0.143, 'train_loss': 0.73837142, 'epoch': 1.0, 'global_step/max_steps': '464/464', 'percentage': '100.00%', 'elapsed_time': '54m 14s', 'remaining_time': '0s'}


[INFO:swift] best_model_checkpoint: /mnt/workspace/output_lora/qwen2_5-0_5b-instruct/v0-20241207-203247/checkpoint-464
[INFO:swift] images_dir: /mnt/workspace/output_lora/qwen2_5-0_5b-instruct/v0-20241207-203247/images
[INFO:swift] End time of running main: 2024-12-07 21:27:49.194640
